# imports ...

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
import time
import os

from datetime import date
from datetime import timedelta
from datetime import datetime

import pickle

import json

import pandas as pd
import numpy as np
import random 



from backtesting import Strategy
from backtesting import Backtest

import pandas_ta as ta
import jinja2

Loading BokehJS ...

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

In [4]:
from handle_data.read_data import load_candles
from backtesting_sampler.sampler import process_heatmap

# Configuration

In [5]:
# %%script false --no-raise-error

if True:
    symbol = "BTC/USDT"
    asset_factor = 1_000_000
    total_cash = 1000
    dca_budget = [50,100,200,400,500,1000]

if False:
    symbol = "ETH/USDT"
    asset_factor = 10_000
    total_cash = 1000
    dca_budget = [50,100,200,400,500,1000]


timeframe='1h'
exchange='Binance'

experiment_nickname = "Up_and_down"

In [6]:
history_days = 21
start_date = (date.today() - timedelta(days=history_days)).strftime('%Y-%m-%d')
end_date=None

if experiment_nickname == "Up_and_down":
    start_date = "2019-11-01"
    end_date="2022-01-23"
elif experiment_nickname == "LongTerm":
    start_date = "2017-01-01"
    end_date=None
elif experiment_nickname == "Up":
    start_date = "2023-01-01"
    end_date=None
elif experiment_nickname == "Bear":
    start_date = "2021-11-12"
    end_date="2022-11-12"
elif experiment_nickname == "Range":
    start_date = "2022-06-16"
    end_date="2023-03-10"

    
if not end_date:
    end_date=date.today().strftime('%Y-%m-%d')
    
print("start_date:",start_date)
print("end_date:",end_date)

start_date: 2019-11-01
end_date: 2022-01-23


In [7]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H%M')
expariment_date = datetime.now().strftime('%Y-%m-%d')
base_experiment_name=(exchange + '_' + symbol + '_' + timeframe + '_' + start_date + '_' + end_date + '_' + timestamp).replace('/', '-')

In [8]:
data = load_candles(exchange, symbol, timeframe=timeframe, factor=asset_factor, start_date=start_date, end_date=end_date)

reading /home/jovyan/work/data/BTC-USDT/1h/BTC-USDT_1h_Binance.pickle


# Class

In [9]:
class DCA_Conditional_Buy_LR_with_TrailingStop(Strategy):  
    # DCA + Moving Stop Loss
    
    band_length = 13
    band_mult = 1
    band_low_pct = 1
    
    intial_sl_factor = 0.0
    minimal_benefit_to_start_trailing = 1.003
    
    sleep_by = 0

    reinvest = 2 # 0 = use only initial, 1 = reinvest all, 2 = use dca_budget reinvesting all
    stop_loose_changes = 0
    i_cash = 1_000
    
    buy_all_days = 0
    dca_budget = 100
    lr_buy_longitude = 3
    
    
    def lb_close(self,data): #lower_band_stdev_formula
        dema = ta.dema(close = data.Close.s, length=self.band_length).to_numpy()
        stdev = ta.stdev(close = data.Close.s, length=self.band_length).to_numpy()
        lower_band = dema - ( self.band_mult * stdev )
        return lower_band
    
    
    def lb_low(self,data): # lower_band_stdev_formula_on_low
        dema = ta.dema(close = data.Close.s, length=self.band_length).to_numpy()
        stdev = ta.stdev(close = data.Close.s, length=self.band_length).to_numpy()
        lower_band = (dema - ( self.band_mult * stdev )) * ( 1 - (self.band_low_pct/100) )
        return lower_band

    
    def LR(self,data):
        # https://greyhoundanalytics.com/blog/custom-indicators-in-backtestingpy/
        linreg = ta.linreg(close = data.Close.s, length=self.lr_buy_longitude, angle=True )
        return linreg.to_numpy()
    

    
    def calculate_size(self):
        if self.reinvest==0:
            if self.equity < self.i_cash:
                cash = self.equity
            else:
                cash = self.i_cash
            size = int((cash / self.data.Close[-1]) * 0.99 )
            #print(f"buy with limit {size} = {self.i_cash} / {self.data.Close[-1]}")
        elif self.reinvest==1:
            size = int( (self.equity / self.data.Close[-1]) * 0.99 )
            #print(f"buy no limit {size} = {self.equity} / {self.data.Close[-1]} ")
        elif self.reinvest==2:
            if self.equity < self.dca_budget:
                cash = self.equity
            else:
                cash = self.dca_budget
            size = int( ( cash / self.data.Close[-1]) * 0.99 )

        return size
    

    
    def init(self):
        self.day_of_week = self.I(lambda x: x,self.data.Close.s.index.dayofweek,plot = False,)
        self.lr_buy = self.I(self.LR, self.data,plot = False)
        self.lower_band = self.I(self.lb_close, self.data)
        self.lb_low = self.I(self.lb_low, self.data)

        self.current_stop_loss = 0 
        self.i_cash = self.equity
        self.dca_bought = False
        
    def do_buy(self,sl):
        self.current_stop_loss = sl * self.intial_sl_factor
        size=self.calculate_size()
        if size>0:
            self.buy(size=size,sl=self.current_stop_loss) 
        
    def next(self):

        new_stop = self.lower_band[-1]
        new_stop_price = self.lb_low[-1]
        
        if self.reinvest in [0,1]:
            if not self.position:
                if not self.should_sleep(): 
                    self.do_buy(new_stop)
                    self.sleep_by = random.random() * 7 * 24 * 60 
        else:
            if self.day_of_week[-1] == 1 or self.buy_all_days: 
                if self.dca_bought == False and self.lr_buy[-1]>0:
                    self.do_buy(new_stop)
                    self.dca_bought == True
            else:
                self.dca_bought == False
        
         
        if new_stop > self.current_stop_loss:
            self.current_stop_loss = new_stop
            for trade in self.trades:
                if new_stop_price > trade.entry_price * self.minimal_benefit_to_start_trailing:
                    old_stop = trade.sl
                    if self.data.Close[-1] > new_stop:
                        trade.sl = new_stop
                        #print(f"I'm recreating the stop loss {old_stop}, new: {new_stop}. Entry price {self.trades[0].entry_price}")
            else:
                #print(f"ERROR, I'm trying to create the stop loss {old_stop}, new: {new_stop}. Entry price {self.trades[0].entry_price}")
                pass

# Optimization

In [10]:
start_time = time.time()

In [11]:
fb = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]

In [12]:
data["open_low_var"] = ((data["Open"] - data["Low"]) / data["Open"]) * 100
open_low_var = data["open_low_var"].mean()
open_low_var

0.5821211500777452

In [13]:
bt = Backtest(data, DCA_Conditional_Buy_LR_with_TrailingStop, cash=total_cash, commission=.001)
optimize = True
heatmap = None

if optimize:
     stats, heatmap = bt.optimize(return_heatmap= True,maximize="Return [%]",
                        band_length=[3,5,9,11,13,21,34],
                        band_mult=[1,2],
                        band_low_pct=open_low_var,
                        #sl_activation_margin=[1.003, 1.004, 1.005, 1.006, 1.007, 1.008, 1.009, 1.01, 1.012 ],
                        minimal_benefit_to_start_trailing=[1.003],
                        reinvest = [2],
                        dca_budget = dca_budget,
                        buy_all_days=[0,1], # daily or weekly
                        lr_buy_longitude=[*range(2, 13, 1)],

    )
else:
    # fast run
    stats = bt.run(band_lenght=11,band_mult=2,minimal_benefit_to_start_trailing=1.003,reinvest=2,dca_budget=200,buy_all_days=1,lr_buy_longitude=6)



Backtest.optimize:   0%|          | 0/11 [00:00<?, ?it/s]

In [14]:
end_time = time.time()
duration = (end_time - start_time)
print("--- %s seconds ---" % duration )

--- 217.86875700950623 seconds ---


# Results

In [15]:
stats._strategy

<Strategy DCA_Conditional_Buy_LR_with_TrailingStop(band_length=21,band_mult=2,band_low_pct=0.5821211500777452,minimal_benefit_to_start_trailing=1.003,reinvest=2,dca_budget=100,buy_all_days=1,lr_buy_longitude=5)>

In [16]:
stats

Start                     2019-11-01 00:00:00
End                       2022-01-23 23:00:00
Duration                    814 days 23:00:00
Exposure Time [%]                   99.554417
Equity Final [$]                  3747.441391
Equity Peak [$]                   7003.449364
Return [%]                         274.744139
Buy & Hold Return [%]              297.177914
Return (Ann.) [%]                   80.698095
Volatility (Ann.) [%]              149.261255
Sharpe Ratio                          0.54065
Sortino Ratio                        1.621809
Calmar Ratio                         1.372352
Max. Drawdown [%]                  -58.802751
Avg. Drawdown [%]                   -3.574427
Max. Drawdown Duration      188 days 10:00:00
Avg. Drawdown Duration        5 days 16:00:00
# Trades                                 1113
Win Rate [%]                        93.890386
Best Trade [%]                      21.542263
Worst Trade [%]                    -48.316551
Avg. Trade [%]                    

In [17]:
#bt.plot()

In [18]:
#from backtesting.lib import plot_heatmaps
#plot_heatmaps(heatmap)

In [19]:
if not(heatmap is None):
    heatmap_df = process_heatmap(heatmap)
else:
    heatmap_df = pd.DataFrame()

heatmap_df

,band_length,band_mult,band_low_pct,minimal_benefit_to_start_trailing,reinvest,dca_budget,buy_all_days,lr_buy_longitude,Return [%]
1488,21,2,0.582121,1.003,2,100,1,5,274.744139
1465,21,2,0.582121,1.003,2,50,1,4,272.079288
1489,21,2,0.582121,1.003,2,100,1,6,271.583166
1490,21,2,0.582121,1.003,2,100,1,7,270.730055
1463,21,2,0.582121,1.003,2,50,1,2,270.689554
...,...,...,...,...,...,...,...,...,...
1308,13,2,0.582121,1.003,2,1000,0,12,51.132569
1,3,1,0.582121,1.003,2,50,0,3,50.950711
2,3,1,0.582121,1.003,2,50,0,4,49.545281
1039,11,2,0.582121,1.003,2,1000,0,7,38.537751


# Write results

In [20]:
base_results_dir = 'results'

In [21]:
srt_strategy_configuration = str(stats._strategy)
str_strategy_name = srt_strategy_configuration[:srt_strategy_configuration.find("(")]
parameters = srt_strategy_configuration[srt_strategy_configuration.find("(") + 1:-1]

In [22]:
directory = base_results_dir + '/' + str_strategy_name + '/' + expariment_date + '/' + experiment_nickname + '/' + base_experiment_name
directory

'results/DCA_Conditional_Buy_LR_with_TrailingStop/2024-02-04/Up_and_down/Binance_BTC-USDT_1h_2019-11-01_2022-01-23_2024-02-04_2021'

In [23]:
if not os.path.exists(directory):
    os.makedirs(directory)

In [24]:
#bt.plot(filename=directory+'/bt_interactive_plot.html')

In [25]:
file_name = directory + "/heatmap_df"   

with pd.ExcelWriter(file_name + ".xlsx", engine='xlsxwriter') as writer:
    heatmap_df.to_excel(writer, index=False)
heatmap_df.to_markdown(file_name + ".md")

file_name = directory + "/trades"   

with pd.ExcelWriter(file_name + ".xlsx", engine='xlsxwriter') as writer:
    stats._trades.to_excel(writer, index=False)
stats._trades.to_markdown(file_name + ".md")

file_name = directory + "/equity_curve"   

with pd.ExcelWriter(file_name + ".xlsx", engine='xlsxwriter') as writer:
    stats._equity_curve.to_excel(writer, index=False)
stats._equity_curve.to_markdown(file_name + ".md")   

In [26]:
str_stats = str(stats)

In [27]:
heatmap_md = heatmap_df[0:10].to_markdown()

In [28]:
strat_return = stats['Return [%]']
buy_and_hold = stats['Buy & Hold Return [%]']
equity_init = stats._equity_curve[0:1]['Equity'].iloc[0]
equity_final = stats['Equity Final [$]']

In [29]:
jinja2_templates_dir = 'templates'
jinja2_template_file = 'single_result.md'

# write results
filename = f"{directory}/index.md"
environment = jinja2.Environment(loader=jinja2.FileSystemLoader(jinja2_templates_dir))
template = environment.get_template(jinja2_template_file)
rendered = template.render(str_strategy_name=str_strategy_name, experiment_nickname= experiment_nickname,
                           timestamp=timestamp,
                           symbol=symbol, exchange=exchange, 
                           start_date=start_date, end_date=end_date, timeframe=timeframe,
                           str_stats=str_stats, 
                           heatmap_md=heatmap_md, parameters=parameters.split(','), 
                           strat_return=strat_return, buy_and_hold=buy_and_hold, 
                           equity_init=equity_init,
                           equity_final=equity_final,duration=duration)

with open(filename, mode="w", encoding="utf-8") as message:
    message.write(rendered)
    print(f"Wrote: {filename}")

Wrote: results/DCA_Conditional_Buy_LR_with_TrailingStop/2024-02-04/Up_and_down/Binance_BTC-USDT_1h_2019-11-01_2022-01-23_2024-02-04_2021/index.md
